In [15]:
import pandas as pd
import csv
import nltk
import os
from pandas import DataFrame
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from collections import defaultdict

In [31]:
Location = r'C:\Users\Valerio\Documents\GitHub\ADM-Hw3-Group4\Airbnb_Texas_Rentals.csv'
df = pd.read_csv(Location)

In [20]:
df.isnull().sum()

Unnamed: 0                 0
average_rate_per_night    28
bedrooms_count             3
city                       0
date_of_listing            0
description                2
latitude                  34
longitude                 34
title                      3
url                        0
dtype: int64

In [32]:
# Drop the rows with null numbers 
df = df[pd.notnull(df["description"])]
df = df[pd.notnull(df["title"])]


In [33]:
# Create a tsv file for each row
for i in range(len(df)):
    pd.DataFrame(df.iloc[i]).transpose().to_csv('Files/doc_%s.tsv'%i, sep='\t')

In [34]:
tokenizer = RegexpTokenizer(r'\w+')
ps = PorterStemmer()

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Valerio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
# Create an empty set for the vocabulary
voc_set = set()
# And an empty dictionary for storage the words for each document
d = {}

In [37]:
# For every file...
for i in range(len(df)):
    doc = pd.read_csv('Files/doc_%s.tsv'%i, sep='\t')
    # Concatenate the description and title in a string
    words = doc["description"][0] + doc["title"][0]
    # Remove "\\n" and replace with a space
    words = words.replace("\\n"," ")
    # Convert everything to lowercase
    words = words.lower()
    # Removing punctuation
    words = tokenizer.tokenize(words)
    # Stemming
    words = [ps.stem(word) for word in words]
    # Keep only unique words
    words = set(words)
    # Removing stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # No integers
    words = [x for x in words if not (x.isdigit() or x[0].isdigit())]
    # Storage the words in vocabulary set
    voc_set.update(words)
    # Up date the dictionary with key: file name, values: words
    d.update( {i:words})

In [38]:
# As a result of this problem, we have a vocabulary set with unique words
# an a dictionary, with key: number of the document values: a list of all the words (filtered) in the Airbnb post

# Create a vocabulary dictionary from the set dictionary 
voc_dict = {}
voc_list = list(voc_set)
for i in range(len(voc_list)):
        voc_dict[i] = voc_list[i]

In [39]:
# Create the index
index = defaultdict(str)

for key, value in d.items():
    L = list(value)
    for value in L:
        if value in index:
            index[value].append(key)
        else:
            index[value] = [key]

In [99]:
# Do the same procedure to the query that we did on the docs

query = input()
# Convert everything to lowercase
query = query.lower()
# Removing punctuation
query = tokenizer.tokenize(query)
# Stemming
query = [ps.stem(word) for word in query]  
# Removing stopwords
query = [word for word in query if word not in stopwords.words('english')]
# No integers
query = [x for x in query if not (x.isdigit() or x[0].isdigit())]
# Keep only unique words
query = set(query)
query = list(query)

garden


In [100]:
# Each of the querys matches should contain all the words on the list query

querys_matches = []
for word in query:
    if word in index.keys():
        querys_matches.append(set(index[word]))


In [101]:
# As a result we are going to have a list, with all the matches, called inter
k = 0
if len(querys_matches) > 0:
    inter = querys_matches[k]
    k += 1
    for i in range(1, len(querys_matches)):
        inter = inter.intersection(querys_matches[k])
        k += 1

In [102]:
# we create an empty datafrae where we are going to put the query's outcome
outcome = pd.DataFrame(columns=['Unnamed: 0', 'Unnamed: 0.1','average_rate_per_night', 'bedrooms_count', 'City', 'date_of_listing', 'Description','latitude','longitude','Title','Url'])

In [203]:
if inter == set():
    print("No results were found with those characteristics")
else:
    for i in range(len(inter)):
        inter = list(inter)
        x = int(inter[i])
        file = pd.read_csv("Files/doc_%s.tsv"%x, sep="\t") 
        file = file.rename(columns={'title': 'Title'})
        file = file.rename(columns={'description': 'Description'})
        file = file.rename(columns={'city': 'City'})
        file = file.rename(columns={'url': 'Url'})
        cols_of_interest = ["Title", "Description", "City", "Url"]
        file[cols_of_interest].style.hide_index()
        frames = [outcome, file]
        outcome = pd.concat(frames)
    outcome = outcome[['Title','Description','City','Url']]
    outcome

C:\Users\Valerio\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [ ]:
The problem is that professor doesn't want the outcome in this way, but in this other:

In [97]:
outcome = outcome[['Title','Description','City','Url']]


In [91]:
#print query's outcome
outcome

,Title,Description,City,Url
0,Unique Location! Alamo Heights - Designer Insp...,"Stylish, fully remodeled home in upscale NW – ...",San Antonio,https://www.airbnb.com/rooms/17481455?location...
0,Private Room Close to Campus,Private bedroom in a cute little home situated...,Bryan,https://www.airbnb.com/rooms/11839729?location...
0,Top Floor Resort Style Living,"Beautiful stained concrete floors, humidity co...",Seabrook,https://www.airbnb.com/rooms/16465532?location...
0,East Austin Hillside Gem,"Beautiful and modern 3Br, 2.5Ba located minute...",Austin,https://www.airbnb.com/rooms/17555039?location...
0,"The Woodlands, BEAUTIFUL HOME, 1 Floor, 2 BT, ...","Attractions: The Woodlands, incredible views, ...",Spring,https://www.airbnb.com/rooms/13065223?location...
0,Ruby - Tiny House at MLF Community First Village,Ruby is one of our 10+ AirBnB units for rent a...,Austin,https://www.airbnb.com/rooms/17508608?location...
0,Lovely queen bedroom in NW Austin,"My house is close to Lakeline Mall, highways a...",Austin,https://www.airbnb.com/rooms/14821566?location...
0,Garden Cottage in East Austin,Our cozy cottage located in our backyard garde...,Austin,https://www.airbnb.com/rooms/2528473?location=...
0,Secluded Treasure in Heart of City,Escape the bustle of Dallas to your own privat...,Dallas,https://www.airbnb.com/rooms/7047741?location=...
0,"#1 2BR Ricchi Luxury Condo in San Antonio, Texas",We have promo rates until July 15! Key in your...,San Antonio,https://www.airbnb.com/rooms/10783113?location...
